## Предобработка данных при помощи Apache Spark.

В этом ноутбуке реализована функция `tools.prep.prep_data` на Scala и Spark.

**Проверьте, что у вас установлена версия Scala 2.12 или ранее, иначе Spark не установится**

In [1]:
// Тут устанавливается спарк
// Будет много всего загружено
import $ivy.`org.apache.spark::spark-sql:2.4.0`

import $ivy.$                                  

In [2]:
import org.apache.spark.sql.functions

// Чтобы не мусорить логами
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

// Модуль spark.sql
import org.apache.spark.sql._

// Создаем сессию и контекст
val spark = NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()



Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql.functions

// Чтобы не мусорить логами

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

// Создаем сессию и контекст

spark: SparkSession = org.apache.spark.sql.SparkSession@15cab9c8

### Выше по ссылке Spark UI откроется интерфейс Apache Spark, где вы сможете отслеживать свои джобы

**Читаем csv файл**


In [3]:
import spark.implicits._
import org.apache.spark.sql.functions._

val path_to_file = "/Users/gleb/Documents/GitHub/RwAttribution/data/alphabank.csv"

var data = spark.read
        .option("header", "true")
        .option("infer_schema", "true")
        .csv(path_to_file)
        // Если файл в другом формате, например parquet, выше измените .csv на .parquet. 
        // Это же применимо и для остальных форматов

// Выводим схему
data.printSchema()


csv at cmd2.sc:9

1 / 1

root
 |-- client_id: string (nullable = true)
 |-- visit_start_time: string (nullable = true)
 |-- interaction_type: string (nullable = true)
 |-- session_with_conversion: string (nullable = true)
 |-- channel_group: string (nullable = true)
 |-- conversion_time: string (nullable = true)
 |-- min_conversion_time: string (nullable = true)



import spark.implicits._

import org.apache.spark.sql.functions._


path_to_file: String = "/Users/gleb/Documents/GitHub/RwAttribution/data/alphabank.csv"
data: DataFrame = [client_id: string, visit_start_time: string ... 5 more fields]

## Предобработка данных

Ниже написана функция предобработки данных. На вход подается датасет, который был получен из файла, а также названия столбца с названием канала, id, типом взаимодействия. 

Более подробное описание формата входных данных читайте https://github.com/realweb-msk/RwAttribution#readme

In [4]:

def prep_data[T](df: Dataset[T], channel_col: String, client_id_col: String,
                 interaction_type_col: String, sep: String="^",
                verbose: Int=0): (Dataset[Row], Dataset[Row], Dataset[Row]) = {

    val full = (
        df
        .groupBy(client_id_col)
        .agg(concat_ws(sep, array_sort(collect_list(channel_col))).alias("path"))
    ).groupBy("path").agg(count(client_id_col))

    val click = (
        df
        .where("%s == 'Click'".format(interaction_type_col))
        .groupBy(client_id_col)
        .agg(concat_ws(sep, array_sort(collect_list(channel_col))).alias("path"))
    ).groupBy("path").agg(count(client_id_col))

    val view = (
        df
        .where("%s == 'Impression'".format(interaction_type_col))
        .groupBy(client_id_col)
        .agg(concat_ws(sep, array_sort(collect_list(channel_col))).alias("path"))
    ).groupBy("path").agg(count(client_id_col))
    
    

    return (full, click, view)
}


val (full_gr, click_gr, view_gr) = prep_data(data, "channel_group", "client_id", "interaction_type")

defined function prep_data
full_gr: Dataset[Row] = [path: string, count(client_id): bigint]
click_gr: Dataset[Row] = [path: string, count(client_id): bigint]
view_gr: Dataset[Row] = [path: string, count(client_id): bigint]

### Ниже все собирается вместе и пишется в один csv файл

In [5]:
val path_to_write = "some_path" // Заменить на другой путь

full_gr.repartition(1).write.csv(path_to_write)

csv at cmd4.sc:3

12 / 12

csv at cmd4.sc:3

200 / 200

csv at cmd4.sc:3

200 / 200

csv at cmd4.sc:3

1 / 1

path_to_write: String = "some_path"